In [6]:
import numpy as np
import importlib

import fastsim

In [7]:
# Load vehicle
veh_name = "2022_Renault_Zoe_ZE50_R135_camelCase"
veh = fastsim.vehicle.Vehicle(veh_file=f"../resources/vehdb/{veh_name}.csv")

fcEffMap is overriding fcEffType in ../resources/vehdb/2022_Renault_Zoe_ZE50_R135_camelCase.csv


In [8]:
# Load WLTC cycle phases
cyc_low = fastsim.cycle.Cycle(cyc_file_path="../resources/cycles/wltc_class3_low3.csv")
cyc_med = fastsim.cycle.Cycle(cyc_file_path="../resources/cycles/wltc_class3_med3b.csv")
cyc_high = fastsim.cycle.Cycle(cyc_file_path="../resources/cycles/wltc_class3_high3b.csv")
cyc_extrahigh = fastsim.cycle.Cycle(cyc_file_path="../resources/cycles/wltc_class3_extra_high3.csv")
# Generate combined cycle
cyc_combined_dict = fastsim.cycle.concat(
    [
        cyc_low.get_cyc_dict(),
        cyc_med.get_cyc_dict(),
        cyc_high.get_cyc_dict(),
        cyc_extrahigh.get_cyc_dict(),
    ]
)
cyc = fastsim.cycle.Cycle(cyc_dict=cyc_combined_dict)

In [9]:
# Simulate combined cycle (Python)
sim = fastsim.simdrive.SimDriveClassic(cyc, veh)
sim.sim_drive()

In [10]:
i0 = len(cyc_low.time_s)
i1 = i0 + len(cyc_med.time_s)-1
i2 = i1 + len(cyc_high.time_s)-1

low = slice(None, i0)
medium = slice(i0-1, i1)
high = slice(i1-1, i2)
extrahigh = slice(i2-1, None)

In [14]:
combined_Wh_per_km_meas = 177
combined_kWh_per_100km_meas = combined_Wh_per_km_meas / 1000 * 100

kWh_per_100km = sim.electric_kWh_per_mi * 1000/fastsim.params.metersPerMile * 100

#print(convert((sim.roadway_chg_kj + sim.ess_dischg_kj) / 3.6e3 / sum(sim_rust.dist_mi)))

print(f"{kWh_per_100km:.3f}")

print(f"{(kWh_per_100km - combined_kWh_per_100km_meas)/combined_kWh_per_100km_meas * 100:+.3f}%")

17.131
-3.214%


In [26]:
def calc_kWh_per_100km(simdr, phase):
    # Copy arrays/lists to np.array for calculations
    roadway_chg_kw_out_ach = np.array(simdr.roadwayChgKwOutAch[phase])
    dt_s = np.array(simdr.cyc.dt_s[phase])
    # Calculate energy consumption
    roadway_chg_kj = sum(roadway_chg_kw_out_ach * dt_s)
    ess_dischg_kj = - (simdr.soc[phase][-1] - simdr.soc[phase][0]) * simdr.veh.maxEssKwh * 3.6e3
    kWh_per_mi = (roadway_chg_kj + ess_dischg_kj) / 3.6e3 / sum(simdr.distMiles[phase])
    kWh_per_100km = kWh_per_mi * 1000/fastsim.params.metersPerMile * 100
    return kWh_per_100km

def calc_percent_error(simulated, measured):
    return 100 * (simulated - measured) / measured

In [27]:
low_kWh_per_100km_meas = 177 / 1000 * 100
medium_kWh_per_100km_meas = 177 / 1000 * 100
high_kWh_per_100km_meas = 177 / 1000 * 100
extrahigh_kWh_per_100km_meas = 177 / 1000 * 100
combined_kWh_per_100km_meas = 177 / 1000 * 100

In [28]:
low_kWh_per_100km = calc_kWh_per_100km(sim, low)
low_kWh_per_100km_error = calc_percent_error(low_kWh_per_100km, low_kWh_per_100km_meas)

medium_kWh_per_100km = calc_kWh_per_100km(sim, medium)
medium_kWh_per_100km_error = calc_percent_error(medium_kWh_per_100km, medium_kWh_per_100km_meas)

high_kWh_per_100km = calc_kWh_per_100km(sim, high)
high_kWh_per_100km_error = calc_percent_error(high_kWh_per_100km, high_kWh_per_100km_meas)

extrahigh_kWh_per_100km = calc_kWh_per_100km(sim, extrahigh)
extrahigh_kWh_per_100km_error = calc_percent_error(extrahigh_kWh_per_100km, extrahigh_kWh_per_100km_meas)

combined_kWh_per_100km = sim.electric_kWh_per_mi * 1000/fastsim.params.metersPerMile * 100
combined_kWh_per_100km_error = calc_percent_error(combined_kWh_per_100km, combined_kWh_per_100km_meas)

In [29]:
print("SIMULATED")
print("--------------------------------------")
print(f"LOW:        {low_kWh_per_100km: >6.3f} kWh/100km ({low_kWh_per_100km_error:+.3f}%)")
print(f"MEDIUM:     {medium_kWh_per_100km: >6.3f} kWh/100km ({medium_kWh_per_100km_error:+.3f}%)")
print(f"HIGH:       {high_kWh_per_100km: >6.3f} kWh/100km ({high_kWh_per_100km_error:+.3f}%)")
print(f"EXTRA-HIGH: {extrahigh_kWh_per_100km: >6.3f} kWh/100km ({extrahigh_kWh_per_100km_error:+.3f}%)")
print("--------------------------------------")
print(f"COMBINED:   {combined_kWh_per_100km: >6.3f} kWh/100km ({combined_kWh_per_100km_error: >7.3f}%)")

SIMULATED
--------------------------------------
LOW:        14.457 kWh/100km (-18.320%)
MEDIUM:     13.968 kWh/100km (-21.083%)
HIGH:       15.439 kWh/100km (-12.773%)
EXTRA-HIGH: 21.424 kWh/100km (+21.040%)
--------------------------------------
COMBINED:   17.131 kWh/100km ( -3.214%)
